In [ ]:
get-date | Join-String -op 'Last ran: ' -format '{0:u}'
$PSVersionTable.PSVersion | Join-String -op 'Pwsh: '
$jstrModule = @{ 'sep' = ', ' ; 'Prop' = { $_.Name, $_.Version -join ': ' } }

if($PSVersionTable.PSEdition -eq 'Desktop') {
    Import-Module -pass 'Benchpress'
    | Join-string @jStrModule | ft
} else {
    # (currently) the fix for core isn't published, so clone it
    Remove-module BenchPress -ea ignore
    Import-Module -passthru (gi 'G:\2022-github\Pwsh\StartAutomating👨\Benchpress\BenchPress.psd1')
    | Join-string @jStrModule | ft
}

Last ran: 2022-07-23 11:19:49Z
Pwsh: 7.2.5
BenchPress: 1.3.2


In [ ]:
$Config = @{ $Names = 'dev.nin' } 
Measure-Benchmark -Technique @{
    # 'Filtering by the Source Command' = {
    'Gcm -CommandType Alias' = {
        Get-Command -m 'dev.nin' -CommandType Alias
    }
    'Gcm | ? CommandType -eq "alias"' = {
        Get-Command -m 'dev.nin'
        | ? CommandType -eq 'alias'        
    }
    '2Step $All.Where{}' = {
        $allCmd = Get-Command -m 'dev.nin'
        $allCmd.Where({$_.CommandType -eq 'alias'})
    }
    'Inline $All.Where{} ' = {
        @(Get-Command -m 'dev.nin').Where({$_.CommandType -eq 'alias'})
    }
} | sort Time -desc


   Benchmark:  @~3696 Mhz

Technique                       Time            RelativeSpeed Throughput
---------                       ----            ------------- ----------
Gcm | ? CommandType -eq "alias" 00:00:00.578315 4.63x         172.92/s
2Step $All.Where{}              00:00:00.484180 3.88x         206.53/s
Inline $All.Where{}             00:00:00.422791 3.39x         236.52/s
Gcm -CommandType Alias          00:00:00.124810 1x            801.21/s

